In [ ]:
'### MULTIPLEXING TELOSEQ ####
## Output

`wf-teloseq` outputs telomere reads in bam format and telomere length statistics in csv format. The csv *telomere_length files will have telomere length estimation from 
searching a predefined number of telomere units "Telomere_length" and the updated estimation from using a sliding composition graph which is the final column in the csv 
file "Telomere_length2". The latter is now used in all outputs. It also produces an html report to summarise run results. If using Pathway 1, an overall sample telomere 
length estimate is given. Unless Pathway 2 is deactivated, the pipeline also reports a weighted average based on mean telomere lengths of chromosome arm assigned reads. 
This weighted average takes into account the variability of each chromosome arms read number and therefore may have less variance when comparing data sets. Only chromosome arms (or contigs) with at least 10x coverage will be reported.

Pathway 1: With the --skipmapping parameter
```
output
├── execution
│   ├── report.html
│   ├── timeline.html
│   └── trace.txt
├── reference
│   └── reference.fasta
├── Sample
│       ├── plots
│       │     └── Sample_raw_Boxplot_of_Telomere_length.pdf
│       ├── reads
│       │     └── Telomere_reads.fastq
│       └── results
│             └── Sample_raw_Per_Read_telomere_length.csv
├── params.json
├── versions.txt
└── wf-teloseq-report.html

| Filter | Description |
|-|-|
| Applied to all | mapq score default is 4 but user can lower or raise if needed, recommended set 0 for denovo pathway. |
| none | no additional filters are appied so no additional reads removed. |
| low stringency | keep only reads in which the end mapping position is 80 bp beyond last telomere motif. This is to remove short telomere only reads that would not be chromosome arm specific and also could be truncated. |
| high stringency | keep only reads in which the start mapping position is before last telomere motif identification and end mapping position is within 25 bp of cutsite with exception of cutsites beyond 45k as will get very few of these reads.
This is to ensure reads span subtelomere and to limit mismapping and fragmented reads. |

In [ ]:
**30/7/24 MULTIPLEXING TELOSEQ**
cd /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/
conda activate wf-teloseq
export TELOSEQ_OUTDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/output/IE14_Teloseq_Multiplex/IE14_barcode02/
export TELOSEQ_WORKDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/work/IE14_Teloseq_Multiplex/IE14_barcode02/
export NXF_SINGULARITY_CACHEDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/singularity/cache
export TELOSEQ_INPUT=/storage/scratch01/groups/bu/teloseq/dorado/output/IE14_Telose_Multiplexing/IE14_demultiplex/IE14_barcode02.fastq 
export TELOSEQ_SAMPLE=/storage/scratch01/groups/bu/teloseq/dorado/output/IE14_Telose_Multiplexing/IE14_demultiplex/IE14_barcode02.fastq 
srun -c 40 --mem=64000 -t 600 --pty bash -c 'nextflow run main.nf -resume -profile singularity -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT'

**01/08/24 MULTIPLEXING TELOSEQ**
##Run the Demultiplexing Command but keeping bam files:
sbatch -p gpu --gres=gpu:A100:1 -t 300 --mem=32G --wrap "/storage/scratch01/users/mespejo/Dorado/bin/dorado demux /storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_multiplexed_data/IE11_Teloseq_Multiplexing_01_supv56mA2all.bam --barcode-arrangement Teloseq.toml --barcode-sequences Teloseq_adapters.fasta --output-dir /storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE11_Teloseq_demultiplex"


##Trying to align the barcode0X.bam files with Dorado 
sbatch -p gpu --gres=gpu:A100:1 -t 300 --mem=32G --wrap "/storage/scratch01/users/mespejo/Dorado/bin/dorado aligner <index> <input_read_folder> --output-dir /storage/scratch01/groups/bu/teloseq/ont-pipeline/output/IE11_demultiplex_align

**16/09/24 Multiplexing Teloseq**
##.1. Basecalling pod5 files: Doing it as the last time I had an error. Tomás told me that it could be an issue if the input&output folders are the same one. 
$ sbatch -p gpu --gres=gpu:A100:1 -t 300 --mem=32G --wrap "/storage/scratch01/users/mespejo/Dorado/bin/dorado demux /storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_Teloseq_multiplexed_data/IE12_Teloseq_Multiplexing_HEK293T_sg2POT1.bam --barcode-arrangement Teloseq.toml --barcode-sequences Teloseq_adapters.fasta --output-dir /storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_demultiplexed/"

##.2. Transform .bam basecalled file into fastq: Pasamos a fastaq para poder hacer Telo-seq. Igual todo esto no habría hecho falta porque aquí no nos interesa mantener el header. 
$ samtools fastq -T "*" ./IE12_Teloseq_Multiplexing_HEK293T_sg2POT1.bam  > IE12_Teloseq_Multiplexing_HEK293T_sg2POT1.fastq

##.3. Hacemos Teloseq primero para el archivo sin demultiplex y luego para cada uno de los barcodes por separado. 
$ cd /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/
$ conda activate wf-teloseq
$ cd/storage/scratch01/groups/bu/teloseq/ont-pipeline/output/IE12_Teloseq_HEK293T_sg2POT1/
$ export TELOSEQ_WORKDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/work/IE12_Teloseq_HEK293T_sg2POT1/IE12_demultiplexed/
$ export NXF_SINGULARITY_CACHEDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/singularity/cache
$ export TELOSEQ_INPUT=/storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_Teloseq_multiplexed_data/IE12_Teloseq_Multiplexing_HEK293T_sg2POT1.fastq
$ export TELOSEQ_SAMPLE=/storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_Teloseq_multiplexed_data/IE12_Teloseq_Multiplexing_HEK293T_sg2POT1.fastq
srun -c 24 --mem=64000 -t 600 --pty bash -c 'nextflow run main.nf -resume -profile singularity,slurm -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT'

srun -c 24 --mem=64000 -t 600 --pty bash -c 'nextflow run main.nf -resume -profile singularity,slurm -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT --reference /storage/scratch01/groups/bu/teloseq/dimeloseq/reference/hg002v1.1.fasta'
        
##Dio error, posiblemente porque necesitaba más memoria, así que ejecuté de nuevo el pipeline con --mem=128000:
$ srun -c 24 --mem=128000 -t 1200 --pty bash -c 'nextflow run main.nf -resume -profile singularity,slurm -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT'

$ export TELOSEQ_OUTDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/output/IE12_Teloseq_HEK293T_sg2POT1/IE12_demultiplexed
$ export TELOSEQ_WORKDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/work/IE12_Teloseq_HEK293T_sg2POT1/
$ export NXF_SINGULARITY_CACHEDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/singularity/cache
$ export TELOSEQ_INPUT=/storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_demultiplexed/Telo-seq_barcode03.fastq
$ export TELOSEQ_SAMPLE=/storage/scratch01/groups/bu/teloseq/ont-pipeline/data/IE12_Teloseq_HEK293T_sg2POT1/IE12_demultiplexed/Telo-seq_barcode03.fastq
srun -c 24 --mem=128000 -t 1200 --pty bash -c 'nextflow run main02.nf -resume -profile singularity,slurm -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT'


In [ ]:
srun -c 24 --mem=64000 -t 600 --pty bash -c 'nextflow run epi2me-labs/wf-teloseq \
--reference /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/test_data/cut_hg002v1.1.fasta \
-profile singularity \
-work-dir $TELOSEQ_WORKDIR \
--out_dir  $TELOSEQ_OUTDIR \
--fastq $TELOSEQ_INPUT'
srun -c 40 --mem=64000 -t 600 --pty bash -c 'nextflow run main.nf --reference /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/test_data/cut_hg002v1.1.fasta --denovo --mapq 0 -resume -profile singularity -work-dir $TELOSEQ_WORKDIR --out_dir $TELOSEQ_OUTDIR --fastq $TELOSEQ_INPUT'
